In [1]:
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
import pyflagser
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics


In [2]:
G=nx.read_gml("Fully Observable/37busEx.gml")

In [3]:
A = nx.to_numpy_array(G)
print(A)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [4]:
N=list(G.nodes)
print(N)
print(len(N))
E=list(G.edges)
print(E)
print(len(E))

['sourcebus', '799', '709', '775', '701', '702', '705', '713', '703', '727', '730', '704', '714', '720', '742', '712', '706', '725', '707', '724', '722', '708', '733', '732', '731', '710', '735', '736', '711', '741', '740', '718', '744', '734', '737', '738', '728', '729', '799r']
39
[('sourcebus', '799'), ('799', '799r'), ('709', '775'), ('709', '731'), ('709', '708'), ('709', '730'), ('701', '702'), ('701', '799r'), ('702', '705'), ('702', '713'), ('702', '703'), ('705', '742'), ('705', '712'), ('713', '704'), ('703', '727'), ('703', '730'), ('727', '744'), ('704', '714'), ('704', '720'), ('714', '718'), ('720', '707'), ('720', '706'), ('706', '725'), ('707', '724'), ('707', '722'), ('708', '733'), ('708', '732'), ('733', '734'), ('710', '735'), ('710', '736'), ('710', '734'), ('711', '741'), ('711', '740'), ('711', '738'), ('744', '728'), ('744', '729'), ('734', '737'), ('737', '738')]
38


In [5]:
import pickle
with open('Fully Observable/fullanomalydataset.pkl', 'rb') as g:
    P0Data = pickle.load(g)

In [6]:
Class=[]
for i in range(len(P0Data)):
    Class.append(P0Data[i]["Anomaly Check"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

500
271
229


In [7]:
def Average(lst):
    return sum(lst) / len(lst)
AverageVoltage=[]
TimeSeries_Voltage=P0Data[330]["TimeSeries_Voltage"]
Voltage=TimeSeries_Voltage[3]
for y in Voltage:
    AverageVoltage.append(Average(list(y)))
AverageVoltage = [i * 10000-9999 for i in AverageVoltage]
print(AverageVoltage)

[0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211]


In [8]:
N_Senario=len(P0Data)
print(N_Senario)

500


In [9]:
def TimeSeries_Fe(TS_array, F_voltage):
    betti_0=[]
    for k in range(len(TS_array)):
        AverageVoltage=[]
        Voltage=TS_array[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        AverageVoltage = [i * 10000-9999 for i in AverageVoltage]
        for p in range(len(F_voltage)):
            n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
            else:
                b=A[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
            n_active.clear()
    return betti_0

In [ ]:
F_voltage=[0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.7]
betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)

In [ ]:
print(betti)

In [10]:
#F_voltage=[0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.7]
F_voltage=[0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.87,0.86,0.7]
Betti_0=[]
N_Senario=len(P0Data)
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)
    Betti_0.append(betti)

Processing file 499 (100%)

In [ ]:
print(Betti_0)

In [12]:
Vol_fec =np.array(Betti_0)
n=len(Betti_0[0])
Feture=[]
for i in range(n):
    Feture.append("{}".format(i))
#Feture=list(range(9,17))
Vdata = pd.DataFrame(Vol_fec, columns =Feture) 
Vdata.insert(loc=n,column='Class',value=Class)
Vdata.head()

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,Class
0,0,0,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,0
1,0,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,0
3,0,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,1,1,1,1,1,...,0,0,1,1,1,1,1,1,1,0


In [13]:
X=Betti_0
y=Class
X=np.array(X)
y=np.array(y)

In [22]:
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    bst = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)

    # Train the model
    bst.fit(X[train], y[train])
    # Evaluate the model
    y_pred=bst.predict(X[test])
    #Import scikit-learn metrics module for accuracy calculation
    
    # Model Accuracy, how often is the classifier correct?
    acc=metrics.accuracy_score(y[test], y_pred)*100
    print("Accuracy:",acc,"%")
    acc_per_fold.append(acc)

      # Increase fold number
    fold_no = fold_no + 1


Accuracy: 86.0 %
Accuracy: 90.0 %
Accuracy: 84.0 %
Accuracy: 86.0 %
Accuracy: 94.0 %
Accuracy: 86.0 %
Accuracy: 82.0 %
Accuracy: 82.0 %
Accuracy: 82.0 %
Accuracy: 86.0 %


In [23]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using XGBoost:", mean_value)
print("Standard Deviation:", std_dev)

Mean using XGBoost: 85.8
Standard Deviation: 3.823901439919996


In [16]:
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):

    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(X)  
    #X_train = scaler.transform(X_train)  
    # apply same transformation to test data
    #X_test = scaler.transform(X_test)  


    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50,),activation='logistic', learning_rate='constant', learning_rate_init=0.01, random_state=1, max_iter=3000, warm_start=True)
    clf.fit(X[train], y[train])
    # Evaluate the model
    y_pred=clf.predict(X[test])

    # Model Accuracy, how often is the classifier correct?
    acc=metrics.accuracy_score(y[test], y_pred)*100
    print("Accuracy:",acc,"%")
    acc_per_fold.append(acc)

      # Increase fold number
    fold_no = fold_no + 1


Accuracy: 84.0 %
Accuracy: 88.0 %
Accuracy: 88.0 %
Accuracy: 88.0 %
Accuracy: 76.0 %
Accuracy: 90.0 %
Accuracy: 78.0 %
Accuracy: 84.0 %
Accuracy: 84.0 %
Accuracy: 82.0 %


In [17]:
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using MLP:", mean_value)
print("Standard Deviation:", std_dev)

Mean using MLP: 84.2
Standard Deviation: 4.565571644870382


In [25]:
from sklearn.ensemble import RandomForestClassifier
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    clfR=RandomForestClassifier(n_estimators=100,max_depth=3,bootstrap=True)

    # Train the model
    clfR.fit(X[train], y[train])
    # Evaluate the model
    y_pred=clfR.predict(X[test])
    #Import scikit-learn metrics module for accuracy calculation
    from sklearn import metrics
    # Model Accuracy, how often is the classifier correct?
    acc=metrics.accuracy_score(y[test], y_pred)*100
    print("Accuracy:",acc,"%")
    acc_per_fold.append(acc)

      # Increase fold number
    fold_no = fold_no + 1

Accuracy: 86.0 %
Accuracy: 86.0 %
Accuracy: 84.0 %
Accuracy: 90.0 %
Accuracy: 90.0 %
Accuracy: 86.0 %
Accuracy: 80.0 %
Accuracy: 78.0 %
Accuracy: 90.0 %
Accuracy: 88.0 %


In [27]:
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using Randomforest:", mean_value)
print("Standard Deviation:", std_dev)

Mean using Randomforest: 85.8
Standard Deviation: 4.157990967870047


In [ ]:
feature=[]
for i in range(n):
    feature.append("{}".format(i))

X=Vdata[feature] # Features
y=Vdata['Class']   # Labels


from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X)  
#X_train = scaler.transform(X_train)  
# apply same transformation to test data
#X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50,),activation='logistic', learning_rate='constant', learning_rate_init=0.01, random_state=1, max_iter=3000, warm_start=True)

scores = cross_val_score(clf, X, y, cv=10)
print("%0.3f accuracy with a standard deviation of %0.2f" % (scores.mean()*100, scores.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
feature=[]
for i in range(n):
    feature.append("{}".format(i))

X=Vdata[feature] # Features
y=Vdata['Class']   # Labels
#print(X)
clf=RandomForestClassifier(n_estimators=100,max_depth=3,bootstrap=True)
scores = cross_val_score(clf, X, y, cv=10)
print("Random Forets")
print("%0.3f accuracy with a standard deviation of %0.2f" % (scores.mean()*100, scores.std()))